# MicroYOLO - ESP32-CAM icin Ultra Hafif YOLO

Bu notebook, ESP32-CAM'de calisabilecek cok kucuk bir YOLO modeli egitir.

**Ozellikler:**
- Girdi: 96x96x3 RGB
- Model boyutu: < 500KB (INT8)
- YOLO tarz cikti (bbox + sinif)
- 0-9 rakam tespiti

In [ ]:
# GPU kontrolu
import tensorflow as tf
print('TensorFlow:', tf.__version__)
print('GPU:', tf.config.list_physical_devices('GPU'))

In [ ]:
# Gerekli kutuphaneler
import numpy as np
import os
import random
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

# Model ayarlari
IMG_SIZE = 96
GRID_SIZE = 6
NUM_CLASSES = 10
OUTPUT_PER_CELL = 5 + NUM_CLASSES  # 15

# Egitim ayarlari - Guncellendi
BATCH_SIZE = 32
EPOCHS = 150  # Arttirildi
LEARNING_RATE = 0.0005  # Dusuruldu
TRAIN_SAMPLES = 15000  # Arttirildi
VAL_SAMPLES = 3000  # Arttirildi

print(f'Grid: {GRID_SIZE}x{GRID_SIZE}, Output per cell: {OUTPUT_PER_CELL}')
print(f'Train: {TRAIN_SAMPLES}, Val: {VAL_SAMPLES}, Epochs: {EPOCHS}, LR: {LEARNING_RATE}')

In [ ]:
# MNIST veri setini yukle
from tensorflow.keras.datasets import mnist
import cv2

(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()
print(f'MNIST Train: {mnist_x_train.shape}')
print(f'MNIST Test: {mnist_x_test.shape}')

# Her sinif icin ornekleri ayir (hizli erisim icin)
mnist_by_class = {}
for i in range(10):
    mnist_by_class[i] = mnist_x_train[mnist_y_train == i]
    print(f'  Sinif {i}: {len(mnist_by_class[i])} ornek')

In [ ]:
def generate_sample_mnist(img_size=96, max_digits=3):
    """MNIST rakamlari kullanarak goruntu ve YOLO etiket olusturur."""
    
    # Arka plan (beyaz veya acik gri + hafif gurultu)
    bg_color = random.randint(220, 255)
    img = np.full((img_size, img_size), bg_color, dtype=np.uint8)
    
    # Hafif gurultu
    noise = np.random.normal(0, 3, img.shape).astype(np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    
    labels = []
    num_digits = random.randint(1, max_digits)
    occupied = []
    
    for _ in range(num_digits):
        digit = random.randint(0, 9)
        
        # MNIST'ten rastgele bir rakam sec
        mnist_idx = random.randint(0, len(mnist_by_class[digit]) - 1)
        digit_img = mnist_by_class[digit][mnist_idx].copy()
        
        # Rastgele boyut (20-40 piksel)
        new_size = random.randint(20, 40)
        digit_img = cv2.resize(digit_img, (new_size, new_size), interpolation=cv2.INTER_AREA)
        
        # Rastgele donus (hafif)
        angle = random.uniform(-15, 15)
        M = cv2.getRotationMatrix2D((new_size/2, new_size/2), angle, 1.0)
        digit_img = cv2.warpAffine(digit_img, M, (new_size, new_size), borderValue=0)
        
        # Cakismayan konum bul
        margin = 3
        for attempt in range(30):
            x = random.randint(margin, img_size - new_size - margin)
            y = random.randint(margin, img_size - new_size - margin)
            
            overlap = False
            for ox, oy, ow, oh in occupied:
                if (x < ox + ow + 2 and x + new_size > ox - 2 and 
                    y < oy + oh + 2 and y + new_size > oy - 2):
                    overlap = True
                    break
            if not overlap:
                break
        
        if overlap:
            continue
        
        occupied.append((x, y, new_size, new_size))
        
        # Rakamin bounding box'ini bul (sifir olmayan pikseller)
        coords = np.where(digit_img > 30)
        if len(coords[0]) > 0:
            y_min, y_max = coords[0].min(), coords[0].max()
            x_min, x_max = coords[1].min(), coords[1].max()
            actual_w = x_max - x_min + 1
            actual_h = y_max - y_min + 1
        else:
            x_min, y_min = 0, 0
            actual_w, actual_h = new_size, new_size
        
        # Rakam rengini tersle (MNIST siyah uzerine beyaz, biz beyaz uzerine siyah istiyoruz)
        digit_inverted = 255 - digit_img
        
        # Rastgele yogunluk
        intensity = random.uniform(0.7, 1.0)
        digit_inverted = (digit_inverted * intensity).astype(np.uint8)
        
        # Goruntiye yerlestir (alpha blending)
        for dy in range(new_size):
            for dx in range(new_size):
                if digit_img[dy, dx] > 30:  # Sadece rakam pikselleri
                    alpha = digit_img[dy, dx] / 255.0
                    img[y + dy, x + dx] = int(img[y + dy, x + dx] * (1 - alpha) + 
                                               digit_inverted[dy, dx] * alpha)
        
        # YOLO formati (normalize edilmis)
        bbox_x = x + x_min
        bbox_y = y + y_min
        x_center = (bbox_x + actual_w / 2) / img_size
        y_center = (bbox_y + actual_h / 2) / img_size
        width = actual_w / img_size
        height = actual_h / img_size
        
        # Sinirlari kontrol et
        x_center = np.clip(x_center, 0, 1)
        y_center = np.clip(y_center, 0, 1)
        width = np.clip(width, 0.05, 1)
        height = np.clip(height, 0.05, 1)
        
        labels.append([digit, x_center, y_center, width, height])
    
    # Grayscale'den RGB'ye
    img_rgb = np.stack([img, img, img], axis=-1)
    
    # Rastgele parlaklik/kontrast
    brightness = random.uniform(0.85, 1.15)
    img_rgb = np.clip(img_rgb * brightness, 0, 255).astype(np.uint8)
    
    return img_rgb, labels

def labels_to_yolo_output(labels, grid_size=6, num_classes=10):
    """YOLO etiketlerini grid formatina donusturur."""
    output = np.zeros((grid_size, grid_size, 5 + num_classes), dtype=np.float32)
    
    for label in labels:
        class_id, x_center, y_center, width, height = label
        grid_x = min(int(x_center * grid_size), grid_size - 1)
        grid_y = min(int(y_center * grid_size), grid_size - 1)
        
        x_offset = x_center * grid_size - grid_x
        y_offset = y_center * grid_size - grid_y
        
        if output[grid_y, grid_x, 4] == 0:
            output[grid_y, grid_x, 0] = x_offset
            output[grid_y, grid_x, 1] = y_offset
            output[grid_y, grid_x, 2] = width
            output[grid_y, grid_x, 3] = height
            output[grid_y, grid_x, 4] = 1.0
            output[grid_y, grid_x, 5 + int(class_id)] = 1.0
    
    return output

print('Fonksiyonlar tanimlandi (MNIST tabanli).')

In [ ]:
# Ornek MNIST goruntuleri goster
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for ax in axes.flatten():
    img, labels = generate_sample_mnist()
    ax.imshow(img)
    ax.axis('off')
    for label in labels:
        cls, xc, yc, w, h = label
        x1 = int((xc - w/2) * IMG_SIZE)
        y1 = int((yc - h/2) * IMG_SIZE)
        rect = plt.Rectangle((x1, y1), int(w*IMG_SIZE), int(h*IMG_SIZE), fill=False, color='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x1, y1-2, str(int(cls)), color='red', fontsize=10, weight='bold')
plt.suptitle('MNIST Tabanli Ornek Egitim Verileri')
plt.tight_layout()
plt.show()

In [ ]:
# MNIST tabanli veri seti olustur
print('Egitim verisi olusturuluyor (MNIST)...')
X_train, y_train = [], []
for i in range(TRAIN_SAMPLES):
    if i % 2000 == 0: print(f'  {i}/{TRAIN_SAMPLES}')
    img, labels = generate_sample_mnist(IMG_SIZE, max_digits=3)
    X_train.append(img)
    y_train.append(labels_to_yolo_output(labels, GRID_SIZE, NUM_CLASSES))

X_train = np.array(X_train, dtype=np.float32) / 255.0
y_train = np.array(y_train, dtype=np.float32)

print('Validation verisi olusturuluyor (MNIST)...')
X_val, y_val = [], []
for i in range(VAL_SAMPLES):
    if i % 500 == 0: print(f'  {i}/{VAL_SAMPLES}')
    img, labels = generate_sample_mnist(IMG_SIZE, max_digits=3)
    X_val.append(img)
    y_val.append(labels_to_yolo_output(labels, GRID_SIZE, NUM_CLASSES))

X_val = np.array(X_val, dtype=np.float32) / 255.0
y_val = np.array(y_val, dtype=np.float32)

print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')

In [ ]:
# MicroYOLO Model - Lambda layer olmadan
class YOLOOutputLayer(layers.Layer):
    """YOLO ciktisi icin ozel aktivasyon layer'i"""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def call(self, x):
        # x shape: (batch, grid, grid, 15)
        # [0:4] -> bbox (x, y, w, h) -> sigmoid
        # [4] -> confidence -> sigmoid
        # [5:15] -> classes -> softmax
        bbox_conf = tf.sigmoid(x[..., :5])
        classes = tf.nn.softmax(x[..., 5:])
        return tf.concat([bbox_conf, classes], axis=-1)
    
    def compute_output_shape(self, input_shape):
        return input_shape

def create_micro_yolo(input_shape=(96, 96, 3), grid_size=6, num_classes=10):
    inputs = keras.Input(shape=input_shape)
    
    # Backbone - cok basit
    x = layers.Conv2D(16, 3, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(2)(x)  # 48x48
    
    x = layers.Conv2D(32, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(2)(x)  # 24x24
    
    x = layers.Conv2D(64, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(2)(x)  # 12x12
    
    x = layers.Conv2D(128, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(2)(x)  # 6x6
    
    # Detection Head
    x = layers.Conv2D(64, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    output_channels = 5 + num_classes
    x = layers.Conv2D(output_channels, 1, padding='same')(x)
    
    # Ozel YOLO aktivasyon layer'i
    outputs = YOLOOutputLayer()(x)
    
    return Model(inputs, outputs, name='MicroYOLO')

model = create_micro_yolo()
model.summary()
print(f'Toplam parametre: {model.count_params():,}')

In [ ]:
# YOLO Loss - Duzeltilmis (Cross-Entropy ile)
def yolo_loss(y_true, y_pred):
    obj_mask = y_true[..., 4:5]  # (batch, 6, 6, 1)
    noobj_mask = 1.0 - obj_mask
    
    # Kayip agirliklari
    lambda_coord = 5.0
    lambda_noobj = 0.5
    lambda_class = 1.0
    
    # Koordinat kaybi (MSE, sadece nesne olan hucrelerde)
    xy_loss = tf.reduce_sum(obj_mask * tf.square(y_true[..., :2] - y_pred[..., :2]))
    
    # Boyut kaybi (sqrt ile)
    wh_true = tf.sqrt(tf.abs(y_true[..., 2:4]) + 1e-6)
    wh_pred = tf.sqrt(tf.abs(y_pred[..., 2:4]) + 1e-6)
    wh_loss = tf.reduce_sum(obj_mask * tf.square(wh_true - wh_pred))
    
    # Confidence kaybi (Binary Cross-Entropy)
    conf_pred = tf.clip_by_value(y_pred[..., 4:5], 1e-7, 1.0 - 1e-7)
    conf_loss_obj = -tf.reduce_sum(obj_mask * tf.math.log(conf_pred))
    conf_loss_noobj = -tf.reduce_sum(noobj_mask * tf.math.log(1.0 - conf_pred))
    
    # Sinif kaybi (Categorical Cross-Entropy, sadece nesne olan hucrelerde)
    class_true = y_true[..., 5:]  # one-hot
    class_pred = tf.clip_by_value(y_pred[..., 5:], 1e-7, 1.0 - 1e-7)  # softmax output
    class_loss = -tf.reduce_sum(obj_mask * class_true * tf.math.log(class_pred))
    
    # Toplam kayip
    total = (lambda_coord * (xy_loss + wh_loss) + 
             conf_loss_obj + lambda_noobj * conf_loss_noobj + 
             lambda_class * class_loss)
    
    return total / tf.cast(tf.shape(y_true)[0], tf.float32)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=yolo_loss)
print('Model derlendi (Cross-Entropy loss ile).')

In [ ]:
# Egitim
callbacks = [
    ModelCheckpoint('micro_yolo_best.keras', monitor='val_loss', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

print('Egitim basliyor...')
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=callbacks, verbose=1)

# Grafik
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
plt.title('Egitim Kaybi')
plt.show()

In [ ]:
# Test - tahmin decode
def decode_predictions(pred, conf_threshold=0.3, img_size=96, grid_size=6):
    detections = []
    for gy in range(grid_size):
        for gx in range(grid_size):
            conf = pred[gy, gx, 4]
            if conf > conf_threshold:
                x_offset, y_offset = pred[gy, gx, 0], pred[gy, gx, 1]
                w, h = pred[gy, gx, 2], pred[gy, gx, 3]
                
                x_center = (gx + x_offset) / grid_size
                y_center = (gy + y_offset) / grid_size
                
                x1 = int((x_center - w/2) * img_size)
                y1 = int((y_center - h/2) * img_size)
                x2 = int((x_center + w/2) * img_size)
                y2 = int((y_center + h/2) * img_size)
                
                class_probs = pred[gy, gx, 5:]
                class_id = np.argmax(class_probs)
                final_conf = conf * class_probs[class_id]
                
                if final_conf > conf_threshold:
                    detections.append([class_id, final_conf, x1, y1, x2, y2])
    return detections

# Sinif basina dogruluk hesapla
print("Sinif basina test yapiliyor...")
class_correct = {i: 0 for i in range(10)}
class_total = {i: 0 for i in range(10)}

for _ in range(500):
    img, labels = generate_sample_mnist(max_digits=1)  # Tek rakam
    if len(labels) == 0:
        continue
    true_class = int(labels[0][0])
    class_total[true_class] += 1
    
    pred = model.predict(np.expand_dims(img/255.0, 0), verbose=0)[0]
    dets = decode_predictions(pred, 0.2)
    
    if len(dets) > 0:
        pred_class = int(dets[0][0])
        if pred_class == true_class:
            class_correct[true_class] += 1

print("\nSinif Basina Dogruluk:")
for i in range(10):
    if class_total[i] > 0:
        acc = class_correct[i] / class_total[i] * 100
        print(f"  Rakam {i}: {class_correct[i]}/{class_total[i]} = {acc:.1f}%")

# MNIST test goster
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for ax in axes.flatten():
    img, labels = generate_sample_mnist()
    pred = model.predict(np.expand_dims(img/255.0, 0), verbose=0)[0]
    dets = decode_predictions(pred, 0.3)
    ax.imshow(img); ax.axis('off')
    
    # Gercek etiketler (yesil)
    for label in labels:
        cls, xc, yc, w, h = label
        x1 = int((xc - w/2) * IMG_SIZE)
        y1 = int((yc - h/2) * IMG_SIZE)
        rect = plt.Rectangle((x1, y1), int(w*IMG_SIZE), int(h*IMG_SIZE), 
                              fill=False, color='green', linewidth=1, linestyle='--')
        ax.add_patch(rect)
    
    # Tahminler (kirmizi)
    for det in dets:
        cls, conf, x1, y1, x2, y2 = det
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, color='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x1, y1-2, f'{cls}:{conf:.0%}', color='red', fontsize=9, weight='bold')
plt.suptitle('Yesil=Gercek, Kirmizi=Tahmin')
plt.tight_layout()
plt.show()

In [ ]:
# TFLite donusumu - Daha iyi quantization
model = keras.models.load_model('micro_yolo_best.keras', 
                                 custom_objects={'yolo_loss': yolo_loss, 'YOLOOutputLayer': YOLOOutputLayer})

# Daha kapsamli representative dataset (her siniftan esit ornek)
def representative_dataset_balanced():
    """Her siniftan esit sayida ornek iceren representative dataset"""
    samples_per_class = 100
    for digit in range(10):
        for _ in range(samples_per_class):
            # Tek rakamli goruntu olustur
            img = np.full((IMG_SIZE, IMG_SIZE), random.randint(220, 255), dtype=np.uint8)
            mnist_digit = random.choice(mnist_by_class[digit])
            
            # Rasgele konum ve boyut
            scale = random.uniform(0.3, 0.6)
            new_h = int(28 * scale * IMG_SIZE / 28)
            new_w = new_h
            
            digit_resized = cv2.resize(mnist_digit, (new_w, new_h))
            digit_resized = 255 - digit_resized  # Invert
            
            max_x = IMG_SIZE - new_w
            max_y = IMG_SIZE - new_h
            x = random.randint(0, max(0, max_x))
            y = random.randint(0, max(0, max_y))
            
            # Blend
            roi = img[y:y+new_h, x:x+new_w]
            mask = digit_resized.astype(np.float32) / 255.0
            blended = (roi * (1 - mask) + 0 * mask).astype(np.uint8)
            img[y:y+new_h, x:x+new_w] = blended
            
            # RGB'ye cevir ve normalize et
            img_rgb = np.stack([img, img, img], axis=-1).astype(np.float32) / 255.0
            yield [np.expand_dims(img_rgb, 0)]

# INT8 Quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_balanced
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

print('TFLite donusumu (INT8 - dengeli dataset)...')
tflite_model = converter.convert()

with open('micro_yolo_int8.tflite', 'wb') as f:
    f.write(tflite_model)

print(f'TFLite model: {len(tflite_model) / 1024:.1f} KB')

# ========== INT8 MODEL DOGRULAMA ==========
print('\n--- INT8 Model Test ---')
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f'Input: {input_details[0]["shape"]}, dtype={input_details[0]["dtype"]}')
print(f'Output: {output_details[0]["shape"]}, dtype={output_details[0]["dtype"]}')

# Her sinif icin test
int8_correct = {i: 0 for i in range(10)}
int8_total = {i: 0 for i in range(10)}

input_scale = input_details[0]['quantization'][0]
input_zp = input_details[0]['quantization'][1]
output_scale = output_details[0]['quantization'][0]
output_zp = output_details[0]['quantization'][1]

print(f'Input quant: scale={input_scale}, zp={input_zp}')
print(f'Output quant: scale={output_scale}, zp={output_zp}')

for digit in range(10):
    for _ in range(30):
        img, labels = generate_sample_mnist(max_digits=1)
        if len(labels) == 0:
            continue
        
        # img 2D (96,96) olmali
        if len(img.shape) == 2:
            img_rgb = np.stack([img, img, img], axis=-1)  # (96, 96, 3)
        else:
            img_rgb = img  # zaten 3D ise
        
        # Normalize ve quantize
        img_float = img_rgb.astype(np.float32) / 255.0
        img_int8 = ((img_float / input_scale) + input_zp).astype(np.int8)
        
        # Batch dimension ekle: (96,96,3) -> (1,96,96,3)
        input_data = img_int8.reshape(1, 96, 96, 3)
        
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        
        output = interpreter.get_tensor(output_details[0]['index'])[0]
        output_scale = output_details[0]['quantization'][0]
        output_zp = output_details[0]['quantization'][1]
        output_float = (output.astype(np.float32) - output_zp) * output_scale
        
        # En yuksek confidence'li hucreyi bul
        best_conf = -1
        best_cls = -1
        for gy in range(6):
            for gx in range(6):
                conf = output_float[gy, gx, 4]
                if conf > best_conf:
                    best_conf = conf
                    class_probs = output_float[gy, gx, 5:15]
                    best_cls = np.argmax(class_probs)
        
        true_cls = int(labels[0][0])
        int8_total[true_cls] += 1
        if best_cls == true_cls:
            int8_correct[true_cls] += 1

print('\nINT8 Model Sinif Basina Dogruluk:')
for i in range(10):
    if int8_total[i] > 0:
        acc = int8_correct[i] / int8_total[i] * 100
        print(f'  Rakam {i}: {int8_correct[i]}/{int8_total[i]} = {acc:.1f}%')

# Bilgileri goster
interp = tf.lite.Interpreter(model_path='micro_yolo_int8.tflite')
interp.allocate_tensors()
inp = interp.get_input_details()[0]
out = interp.get_output_details()[0]
print(f"Input: {inp['shape']}, scale={inp['quantization'][0]:.6f}, zp={inp['quantization'][1]}")
print(f"Output: {out['shape']}, scale={out['quantization'][0]:.6f}, zp={out['quantization'][1]}")

In [ ]:
# C Header dosyasi olustur
def create_c_header(tflite_path, header_path):
    with open(tflite_path, 'rb') as f:
        data = f.read()
    
    interp = tf.lite.Interpreter(model_path=tflite_path)
    interp.allocate_tensors()
    inp = interp.get_input_details()[0]
    out = interp.get_output_details()[0]
    
    with open(header_path, 'w') as f:
        f.write('// MicroYOLO for ESP32-CAM - Auto-generated\n')
        f.write(f'// Model size: {len(data)} bytes\n\n')
        f.write('#ifndef MICRO_YOLO_MODEL_H\n#define MICRO_YOLO_MODEL_H\n\n')
        f.write('#include <stdint.h>\n\n')
        
        f.write('// Sabitler\n')
        f.write('#define MICRO_YOLO_INPUT_SIZE 96\n')
        f.write('#define MICRO_YOLO_GRID_SIZE 6\n')
        f.write('#define MICRO_YOLO_NUM_CLASSES 10\n\n')
        
        f.write('// Quantization\n')
        f.write(f'const float input_scale = {inp["quantization"][0]}f;\n')
        f.write(f'const int input_zero_point = {inp["quantization"][1]};\n')
        f.write(f'const float output_scale = {out["quantization"][0]}f;\n')
        f.write(f'const int output_zero_point = {out["quantization"][1]};\n\n')
        
        f.write('const char* digit_labels[] = {"0","1","2","3","4","5","6","7","8","9"};\n')
        f.write('const int NUM_CLASSES = 10;\n\n')
        
        f.write(f'const unsigned int digit_model_len = {len(data)};\n')
        f.write('alignas(8) const unsigned char digit_model[] = {\n')
        
        for i, byte in enumerate(data):
            if i % 16 == 0: f.write('  ')
            f.write(f'0x{byte:02x},')
            if i % 16 == 15: f.write('\n')
        
        f.write('\n};\n\n#endif\n')
    
    print(f'Header: {header_path} ({len(data)/1024:.1f} KB)')

create_c_header('micro_yolo_int8.tflite', 'micro_yolo_model.h')

In [ ]:
# Dosyalari indir
from google.colab import files
print('Dosyalar indiriliyor...')
files.download('micro_yolo_model.h')
files.download('micro_yolo_int8.tflite')
print('Tamamlandi! micro_yolo_model.h dosyasini ESP32 projesine kopyala.')